## 데이터 준비

Image classification을 위한 데이터셋 예시 (Wonders of the World Image Dataset)
- Kaggle : https://www.kaggle.com/datasets/balabaskar/wonders-of-the-world-image-classification?resource=download

해당 이미지 데이터는 kaggle 계정으로 로그인 후 다운로드 받을 수 있습니다.

### 이미지 데이터 준비

기존의 구조

```
image-dir
  - class1
    - img1.jpg
    - img2.jpg
      ...
  - class2
    - img1.jpg
    - img2.jpg
    
...
```

이것을 train/test 데이터를 10% 비율 정도로 나누어서 s3에 올려놓도록 합니다.
만일 이미 train/test가 나뉜 데이터를 받았다면, 아래에 dataset split 하는 부분은 진행하지 않고 s3에 업로드만 진행합니다.


In [ ]:
import os
import random
import shutil

In [ ]:

def copy_data(src_path, dst_path):
    os.makedirs(os.path.dirname(dst_path), exist_ok=True)
    shutil.copyfile(src_path, dst_path)

def split_image_dataset(image_data_dir, output_dir):
    label_list = os.listdir(image_data_dir)
    
    for label in label_list:
        label_dir = os.path.join(image_data_dir, label)
        image_list = os.listdir(label_dir)
        
        random.shuffle(image_list)
        ratio = int(len(image_list) / 10)
        train_data = image_list[ratio:]
        test_data = image_list[:ratio]
        assert len(image_list) == len(train_data) + len(test_data)
        assert len(train_data) > len(test_data)
        
        for train_image in train_data:
            current_path = os.path.join(image_data_dir, label, train_image)
            copy_path = os.path.join(output_dir, "train", label, train_image)
            copy_data(current_path, copy_path)
        
        for test_image in test_data:
            current_path = os.path.join(image_data_dir, label, test_image)
            copy_path = os.path.join(output_dir, "test", label, test_image)
            copy_data(current_path, copy_path)
            
        print(f"Label [{label}] => train: {len(train_data)}, test: {len(test_data)}")
            

In [ ]:
split_image_dataset("data/wwi", "data/wwi-dataset")

### S3에 데이터 업로드

- SageMaker를 활용할 때 S3는 가장 기본이 되는 스토리지입니다.
- 이미 train/test 나뉜 데이터셋이 준비되었다면 아래와 같이 default bucket에 업로드를 진행합니다.
- jupyter shell에서 실행해도 되지만 terminal 에서 그냥 실행해도 상관 없습니다.

In [ ]:
# !mkdir data; cd data; wget [데이터 경로]; tar zxvf wwi-dataset.tar.gz

In [ ]:
from sagemaker.session import Session

sagemaker_session = Session()
bucket = sagemaker_session.default_bucket()
print(bucket)

In [ ]:
# !aws s3 cp data/wwi-dataset s3://{bucket}/lowcode-sm/wwi-dataset/ --recursive

이제 이미지 데이터셋이 준비되었기 때문에 활용이 가능합니다!

## Time series 데이터 준비

- 아래 데이터의 경우 time-series 테스트를 위해서 필요한 내용입니다.
- 따라서, time-series 실습을 진행하지 않는 경우 아래 내용을 진행 할 필요가 없습니다.

Time series를 위한 데이터셋
- UCI electricty 경로 : `s3://sagemaker-sample-files/datasets/timeseries/uci_electricity/LD2011_2014.txt.zip`

데이터가 없다면 위의 경로에서 데이터를 받은 후 압축을 풀고 아래의 과정을 진행하면 됩니다.
만일 이미 train/test 가 나뉘어 데이터가 준비되어 있다면, 해당 데이터를 받아서 s3에 올려놓으면 됩니다.


In [ ]:
# !cd data; aws s3 cp s3://sagemaker-sample-files/datasets/timeseries/uci_electricity/LD2011_2014.txt.zip . ; mkdir uci-elec; unzip LD2011_2014.txt.zip -d uci-elec

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta

In [ ]:
file_path = "data/uci-elec/LD2011_2014.txt"

In [ ]:
df = pd.read_csv(file_path, sep=";", index_col=0, parse_dates=True, decimal=",")
num_timeseries = df.shape[1]
data_kw = df.resample("2H").sum() / 8
timeseries = []
for i in range(num_timeseries):
    timeseries.append(np.trim_zeros(data_kw.iloc[:, i], trim="f"))

In [ ]:
df

In [ ]:
fig, axs = plt.subplots(5, 2, figsize=(20, 20), sharex=True)
axx = axs.ravel()
for i in range(0, 10):
    timeseries[i].loc["2014-01-01":"2014-01-14"].plot(ax=axx[i])
    axx[i].set_xlabel("date")
    axx[i].set_ylabel("kW consumption")
    axx[i].grid(which="minor", axis="x")

In [ ]:
# we use 2 hour frequency for the time series
freq = "2H"

# we predict for 7 days
prediction_length = 7 * 12

# we also use 7 days as context length, this is the number of state updates accomplished before making predictions
context_length = 7 * 12

In [ ]:
start_dataset = pd.Timestamp("2014-01-01 00:00:00", freq=freq)
end_training = pd.Timestamp("2014-09-01 00:00:00", freq=freq)

In [ ]:
training_data = [
    {
        "start": str(start_dataset),
        "target": ts[
            start_dataset : end_training - timedelta(days=1)
        ].tolist(),  # We use -1, because pandas indexing includes the upper bound
    }
    for ts in timeseries
]
print(len(training_data))

In [ ]:
num_test_windows = 4

test_data = [
    {
        "start": str(start_dataset),
        "target": ts[start_dataset : end_training + timedelta(days=k * prediction_length)].tolist(),
    }
    for k in range(1, num_test_windows + 1)
    for ts in timeseries
]
print(len(test_data))

In [ ]:
def write_dicts_to_file(path, data):
    with open(path, "wb") as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode("utf-8"))

In [ ]:
output_dir = "data/uci-elec-dataset"
os.makedirs(os.path.join(output_dir, "train"))
os.makedirs(os.path.join(output_dir, "test"))
train_path = os.path.join(output_dir, "train", "train.json")
test_path = os.path.join(output_dir, "test", "test.json")

In [ ]:
%%time
write_dicts_to_file(train_path, training_data)
write_dicts_to_file(test_path, test_data)

### 시계열 데이터 S3에 업로드

- 이미 준비된 데이터가 있다면 s3에 올려놓도록 합니다.

In [ ]:
# !mkdir -p data; cd data; wget [데이터 주소]; tar zxvf uci-elec-dataset.tar.gz

In [ ]:
# !aws s3 cp data/uci-elec-dataset s3://{bucket}/lowcode-sm/uci-elec-dataset/ --recursive

## Tabular 데이터 S3에 업로드

Kaggle 의 housing price를 예측하는 regression 용 데이터입니다.
- https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques

In [ ]:
# !mkdir -p data; cd data; mkdir -p hp; wget [데이터 주소]; unzip house-prices-advanced-regression-techniques.zip -d hp

In [ ]:
# !aws s3 cp data/hp s3://{bucket}/lowcode-sm/hp/ --recursive